# Protein isoforms

The analysis the Translation Initiation Site (TIS) profiling data [@Eisenberg2020-bg]. 

In [1]:
## logging functions
import logging
## data functions
import pandas as pd
## system functions
from os.path import dirname
import sys
## system functions from roux
from roux.lib.io import backup
from IPython.display import Markdown as info_nb
from roux.lib.io import read_table
from roux.lib.io import to_dict
## visualization functions
import matplotlib.pyplot as plt
## visualization functions from roux
from roux.viz.io import to_plot
## data functions from roux
import roux.lib.dfs as rd # attributes
sys.path.append('..')

In [2]:
## parameters
metadata_path='../config/metadata.yaml'
force=False
test=True

In [21]:
## inferred parameters
metadata=read_metadata(metadata_path,inputs=None if not test else {'version':{'number':'test'}},)
metadata['dataset']=read_metadata(metadata['dataset_config_path'],config_base=dict(species_name=metadata['species_name'],path=metadata['dataset_path'],),)
### output
output_dir_path=metadata['processed']['mechanisms']
logging.info(f"Output directory: {output_dir_path}")
## backup old files if overwriting (force is True)
if force: backup(output_dir_path,dirname(output_dir_path),test=not force,)

In [11]:
from Bio.Seq import Seq as to_seq
## common functions
def read_fasta(
    fap: str,
    key_type: str='id',
    duplicates: bool=False,
    ) -> dict:
    """Read fasta

    Args:
        fap (str): path
        key_type (str, optional): key type. Defaults to 'id'.
        duplicates (bool, optional): duplicates present. Defaults to False.

    Returns:
        dict: data.

    Notes:
        1. If `duplicates` key_type is set to `description` instead of `id`.
    """
    from Bio import SeqIO
    if (not duplicates) or key_type=='id':
        try:
            id2seq=SeqIO.to_dict(SeqIO.parse(fap,format='fasta'))
            id2seq={k:str(id2seq[k].seq) for k in id2seq}
            return id2seq
        except:
            duplicates=True
    if duplicates or key_type=='description':
        id2seq={}
        for seq_record in SeqIO.parse(fap, "fasta"):
            id2seq[getattr(seq_record,key_type)]=str(seq_record.seq)
        return id2seq

## TIS profiling data

|   |  |
|---|---|
| Growth phase | Vegetative exponential |
| Sample ID | GSM4547703 |
| SRA ID | SRR11777267 |  
| Data access link | https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE150375 |
| Alignment viewer link| https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&page_size=10&acc=SRR11777267&display=alignment |  

In [7]:
import pysam
input_bam_file=f"{metadata['dataset']['external']['path']}/2020_Eisenberg_et_al/SRR11777267_chr13_cue4.bam"
pysam.index(input_bam_file)
aligned=pysam.AlignmentFile(
    input_bam_file,
    'rb')

In [8]:
gene_symbol='CUE4'
coord=dict(
    contig='chr13',
    start=69735,
    end=70088,
    )
seq_offset_up=55
## Eisenberg et al., 2020 GFF co-ordinates 77513:77866

In [12]:
seq_ref=list(read_fasta(f"{metadata['dataset']['external']['path']}/2020_Eisenberg_et_al/genes/{gene_symbol}.fasta").values())[0]

In [13]:
info_nb(f"The sequence of the canonical isoform:\n{str(to_seq(seq_ref).reverse_complement().translate())}")

In [14]:
info_nb(f"The sequence of the non-canonical isoform:\n{str(to_seq(seq_ref).reverse_complement().translate())[19:]}")

In [15]:
info_nb(f"The difference in molecular weight of the isoforms={(12892.78-10757.16)/1e3:.1f}.")

In [16]:
## make a dataframe
df0=pd.DataFrame({'position':list(range(coord['start'],coord['end']+1,1)),
'reference base': list(seq_ref),}).rd.assert_no_na()
df0.head(1)

### Read depth

In [17]:
df01 = pd.DataFrame([x.split('\t') for x in pysam.depth(input_bam_file).split('\n')],
                   columns=['feature','position','depth'],
                   )
df01.head(1)

In [18]:
## clean
df01=df01.log.dropna(subset=['position']).astype({'position':int,'depth':float},)

In [19]:
## filter
df1=df01.log.query(expr=f"`position` > {coord['start']} & `position` < {coord['end']+seq_offset_up}")

The positions are continuous. Only the regions with aligned reads are included.

In [20]:
## merge with the isoform annotations
df2=df0.log.merge(right=df1,
             how='outer',
             validate="1:1",
             )
df2.head(1)

In [32]:
input_path_=to_table(df2,metadata['mechanisms']['non_canonical_isoform']['read_depth'][gene_symbol])

In [45]:
from roux.viz.colors import saturate_color,to_hex

In [48]:
kws_plot=dict(
    gene_symbol=gene_symbol,
    coord=coord,
    seq_offset_up=seq_offset_up,
    features_colors=[to_hex(saturate_color(metadata['colors']['gene2'],0.5)),metadata['colors']['gene2']],
    )
input_config_path_=to_dict(kws_plot,
        Path(metadata['mechanisms']['non_canonical_isoform']['read_depth'][gene_symbol]).with_suffix('.yaml').as_posix(),
       )

## Visualize

In [49]:
from modules.tools.plot import plot_read_depth
plot_read_depth(
    data=read_table(input_path_),
    seq_ref=list(read_fasta(f"{metadata['dataset']['external']['path']}/2020_Eisenberg_et_al/genes/{gene_symbol}.fasta").values())[0],
    **read_dict(input_config_path_),    
    )
plt.tight_layout()
to_plot(
    Path(metadata['mechanisms']['non_canonical_isoform']['read_depth'][gene_symbol]).with_suffix('').as_posix(),
    fmts=['pdf','png'],
)